Control de hojas de google calculo de Google (Googlesheets) con Python en Jupyter Notebooks dentro de VSC.

Configuramos las credenciales dentro de Google Cloud Console partiendo. Creando una cuenta de servicio, habilitando la API de Google Sheets dentro del entorno y posteriormente creando las credenciales para descargar el archivo JSON y poderlo utilizar dentro del entorno VSC.

Al haber configurado las credenciales y las conexiones (API) dentro de Google Cloud Console, tenemos que instalar varias librerias dentro de el entorno python.

Para utilizar las librerias de google tenemos que importar las siguientes dependencias:

**pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib**

las cuales son 3 dependencias en total.

In [79]:
#Importamos librerias
from google.oauth2 import service_account           #Utilizar la cuenta de servicio donde se descargaron las credenciales
from googleapiclient.discovery import build         #Crear el objeto de servicio, que sera nuestro medio de conexión
import gspread                                      #libreria de manipulación de hojas de calculo de google con python
import pandas as pd
import numpy as np                          #librerias de manipulacion de dato

In [2]:
#Creamos las conexiones VSC-GoogleSheets

#Creamos los documentos que necesitaremos para las credenciales

ruta_permisos = "D:/SITEC/permisos/sitecsolucionesyservicios-67cee8fd8c65.json"          #ubicacion del archivo JSON con las credenciales
alcances = ['https://www.googleapis.com/auth/spreadsheets']     #scopes: definen los alcances y capacidades que tendra la conexión

#Creamos las credenciales
#la entrada de la funcion from_service_account_file toma los accesos (archivo JSON) y que tanto alcance tendrán los mismos
credenciales = service_account.Credentials.from_service_account_file(ruta_permisos, scopes = alcances)

#Creamos el objeto que tendra la conexión entre la API google sheets & VSC
#service_drive:

service = build('sheets', 'v4', credentials=credenciales)    #build('ServiceName' , 'Version', credentials = 'ruta_credenciales')


Operaciones básicas de control y manipulacion:
La documentación proviene de Google Workspace para varios lenguajes:

**https://developers.google.com/sheets/api/guides/concepts?hl=es-419**

Tenemos el siguiente recurso:

spreadhseets.values (ValueRange): Datos de  un rango de la hoja de calculo

campos:

|**range (string):** el rango que abarcan los valores en notacion celdas. Indican las celdas que se van a requerir
ej:
-**range = A1:B4**


|**majorDimension (enum (dimension)):** Es la dimension principal de valores, y determinara si los datos se tomaran horizontalmente o verticalmente, las dos posibles opciones son: **ROWS** y **COLUMNS**
ej:

|-**majorDimension = ROWS**  : implica que tomará los valores del rango inicial al final de forma horizontal para este caso.
si no se añade este campo, el valor predeterminado es ROWS.

|**values []**: Los datos que se leyeron o se escribieron, tienen una estructura de array de arrays, el externo representa todos los datos y cada interno representa una dimension principal. cada elemento de la matriz interna corresponde a una celda.
En el resultado, no se incluirán files y columnas finales vacias.
los valores admitidos son: **bool** , **string**, **double**. Se omitiran los valores nulos.
Para establecer un valor vacion en una celda, establece el valor de string en una string vacia



Este recurso tiene varios metodos los cuales se presentarán a continuación:

**append**  ,  **BatchUpdate**  ,  **update()**

Parametros de ruta de acceso:

**spreadsheetId: sp_id** :determina el ID de la hoja de calculo, se puede determinar abriendo la hoja de calculo en una pagina nueva y lo encuentras en el URL, la parte subrayada, ejemplo: 

-https://docs.google.com/spreadsheets/d/**1SElbxT0F96KqY9Q966tUr1WSUjdakH_ew7NtOKu3G0M**/edit#gid=0 


**range** : La notacion de celdas (A1). OJO: *los valores se agregan despues de la ultima fila de la tabla que detecte en ese rango.*

Parametros de consulta:
**valueInputOption** : controla unicamente como se agregaran los datos de entrada a a la hoja (*columnas o filas*)
existen 3 formas:

°INPUT_VALUE_OPTION_UNSPECIFIED : valor de entrada predeterminado

°RAW : los valores que ingresó el usiario no se analizarán y se almacenaran como están

°USER_ENTERED : Los valores se analizarán como si el usuario los escribiera en la UI.



**InsertDataOption** : Determina como se cambian los datos existentes cuando se ingresan datos nuevos.


**includeValuesInResponse** : Determina si la respuesta de actualizacion debe incluir los valores de las celdas que se agregaron. De forma predeterminada, las respuestas no incluyen los valores actualizados. 


**responseValueRenderOption**.

**responseDateTimeRenderOption**.

SCOPES:

°https://www.googleapis.com/auth/drive

°https://www.googleapis.com/auth/drive.file

°https://www.googleapis.com/auth/spreadsheets









In [17]:
#append
hoja = service.spreadsheets()

sp_id = '1IElbxT0F96KqY9Q966tUr1WSUjdakH_ew7NtOKu3G0M'

resappend = hoja.values().append(
                        spreadsheetId =sp_id,
                        range = 'Hoja 1!D1:H1',      #si no se selecciona la hoja, por default sera la primera hoja
                        valueInputOption = 'USER_ENTERED',
                        body = {'values' : [[23,3,3,3,22]]}).execute()    #Si detecta un valor en la celda que rescribiras, marcará error

#Los resultados se guardan en formato diccionario


In [57]:
#Update

nombre_hoja = 'Hoja 1'
range = 'B2'

data = (
        ('columna1','columna2','columna3','columna4'),
                    (1,2,3,4),                              #Es importante denotar que la API de Google espera una lista de listas (lista anidada)
                    (20,21,22,23,24)                        #para los datos, en este caso funciona por que son tupla de tulpas
)                                                           #pero en general la API necesita una lista de listas.
value_range_body_rows = {
    
    'majorDimension': 'ROWS',
    'values' : data
}


value_range_body_columns = {
    'majorDimension' : 'COLUMNS',
    'values' : data
}

resupdate_rows = hoja.values().update(
    spreadsheetId = sp_id,
    range = 'Hoja 2!B4',    #Donde se insertará la data, tenemos que tener cuidado con los rangos de incersion de los datos 
    valueInputOption = 'USER_ENTERED',
    body = value_range_body_rows
).execute()

resupdate_columns = hoja.values().update(
    spreadsheetId = sp_id,
    range = 'Hoja 3!B2',    #Donde se insertará la data, tenemos que tener cuidado con los rangos de incersion de los datos 
    valueInputOption = 'USER_ENTERED',
    body = value_range_body_columns
).execute()

#OTRO EJEMPLO

valores1 = [
    [132,123,345,567,689,2345,245,578,345]
]

valores2 = [
    [1,4,9,4,5,1,4,9,6] 
]
valores3 = [
    ['Hola', 'a', 'todos', 'como', 'estan?', 'los', 'saluda', 'Samael', 'TheCreator']
]

value_range_body = {
    'majorDimension' : 'ROWS',
    'values' : valores3
}

hoja.values().update(
    spreadsheetId = sp_id,
    range = 'Hoja 3!A1:I1',
    valueInputOption = 'USER_ENTERED',
    body = value_range_body
).execute()


In [ ]:
#batchUpdate                            #tiene errores

resbatchupdate = hoja.values().batchUpdate(
                spreadsheetId = sp_id,
                range = 'Hoja 2!A1',        #si no se selecciona la hoja, por default sera la primera hoja
                ValueInputOption = 'USER_ENTERED',
                body = value_range_body_rows
                ).execute()
                

Lectura de información

La lectura de información requiere del modulo siguiente:

**get()**

El modulo get nos ayuda a capturar o extraer información del objeto al que se aplica.
Para este caso tiene mismos parametros que la función append:

|**spreadhseetId= sp_id** : Parametro de ruta, define el Id de la hoja de calculo

|**range=** : define el rango de celdas que vas a capturar o leer los datos.
OJO: *Si no determinas el range, la funcion por default te traera toda la información que hay en la hoja*


In [45]:
#import os
#from google import Create_Service

In [48]:
#get()
lectura = hoja.values().get(
        spreadsheetId= sp_id,
       #majorDimension = 'ROWS' 
        range = 'Hoja 1!A1:A7').execute()   #si no se declara el rango, por default tomará toda la hoja

print(lectura.keys())
print(lectura['range'])
print(lectura['majorDimension'])
print(lectura['values'])

#los resultados se guardan en formato diccionario: donde cada parametro es la clave y los valores el valor.

"""
columns = lectura['values'][0]
data = lectura['values'][1:]
df = pd.DataFrame(data,columns = columns)
"""

dict_keys(['range', 'majorDimension', 'values'])
'Hoja 1'!A1:A7
ROWS
[['1'], ['2'], ['3'], ['prueba'], ['2'], ['2'], ['23']]


In [72]:
#BatchGet() : Muestra uno o más rangos de valores de una hoja de cálculo. El emisor debe especificar el ID de la hoja de cálculo y uno o más rangos

valueRange_body = [
    'Hoja 1!A1:I1',
    'Hoja 2!A1:I1',
    'Hoja 3!A1:I1',
]


lecturabatch = hoja.values().batchGet(
    spreadsheetId = sp_id,
    majorDimension = 'ROWS',
    ranges = valueRange_body
).execute()

print(lecturabatch.keys())

dict_keys(['spreadsheetId', 'valueRanges'])


In [104]:
#Extraer datos y transformarlos a un DataFrame


tabla = hoja.values().get(
    spreadsheetId = sp_id,
    majorDimension = 'ROWS',  
    range = 'Hoja 2!B2:E6'
    ).execute()

#columnas = tabla[]
#data = tabla[1:2]

data = tabla['values'][1:]
columnas = tabla['values'][0]
dataframe = pd.DataFrame(data,columns=columnas)
print(dataframe)

  columna1 columna2 columna3 columna4
0      342      345      123        4
1       21      123       64       75
2        1   213123    23542   142323
3       20   123123     2324      123
